<a href="https://colab.research.google.com/github/swapnilbetkar/python/blob/Dev_Self/Code2_Swapnil_Betkar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
    from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
import plotly.graph_objs as go
import plotly.figure_factory as ff
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [40]:
dataset = pd.read_csv("/content/drive/MyDrive/Datasets/CompletionRates.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300 entries, 0 to 14299
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HIGH_DEG        14300 non-null  int64  
 1   INST_TYPE       14300 non-null  int64  
 2   INST_EXP_PS     14300 non-null  int64  
 3   AVG_FAC_SAL_PM  14300 non-null  int64  
 4   TS              14300 non-null  int64  
 5   TS_MEN          14300 non-null  float64
 6   TS_WOM          14300 non-null  float64
 7   TS_MAR          14300 non-null  float64
 8   TS_DPEN         14300 non-null  float64
 9   TS_VET          14300 non-null  float64
 10  PER_PT          14300 non-null  float64
 11  ST_FI_LO        14300 non-null  float64
 12  ST_FI_M1        14300 non-null  float64
 13  ST_FI_M2        14300 non-null  float64
 14  ST_FI_H1        14300 non-null  float64
 15  ST_FI_H2        14300 non-null  float64
 16  HL_ED_P_MS      14300 non-null  float64
 17  HL_ED_P_HS      14300 non-null 

In [41]:
corrs = dataset.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure.show()

In [42]:
categorical_feature = ['INST_TYPE']
dataset = pd.get_dummies(dataset, columns = categorical_feature)
X = dataset.drop(['TS_MEN','HECR','TS_DPEN','ST_FI_LO','HL_ED_P_PS'], axis = 1) # Features
Y = dataset['HECR'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(14300, 17)
(14300,)


In [46]:
# # Normalizing numerical features so that each feature has mean 0 and variance 1
# feature_scaler = StandardScaler()
# X_scaled = feature_scaler.fit_transform(X)

# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='mse', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [700,750,800,850]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# print("Cross-validation results:\n", pd.DataFrame.from_dict(gd_sr.cv_results_))

# best_parameters = gd_sr.best_params_
# print("Optimal parameters:\n", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("Best mean cross-validated score:\n", best_result)


Cross-validation results:
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0      29.962712      0.168471         0.620901        0.047329   
1      32.098702      0.153272         0.665030        0.047699   
2      34.234306      0.142347         0.713125        0.057498   
3      36.498713      0.286802         0.766289        0.047261   

  param_n_estimators                 params  split0_test_score  \
0                700  {'n_estimators': 700}           0.622326   
1                750  {'n_estimators': 750}           0.622923   
2                800  {'n_estimators': 800}           0.622991   
3                850  {'n_estimators': 850}           0.622650   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.670730           0.720050           0.750170           0.705056   
1           0.670821           0.719812           0.750253           0.705185   
2           0.670788           0.720139           0.750197     

In [47]:
# rfr = RandomForestRegressor(n_estimators=800, criterion='mse', max_features='sqrt', random_state=1)
# rfr.fit(X_scaled,Y)

# ##Creating a feature importance list to identify significant features
# featimp = pd.Series(rfr.feature_importances_, index=list(X)).sort_values(ascending=False)
# print(featimp)

ST_FI_H2          0.152007
HL_ED_P_HS        0.145402
PER_PT            0.114497
INST_EXP_PS       0.098899
AVG_FAC_SAL_PM    0.088253
TS                0.064854
ST_FI_M1          0.060626
TS_MAR            0.047010
TS_WOM            0.043747
ST_FI_H1          0.040017
HL_ED_P_MS        0.038714
ST_FI_M2          0.034518
TS_VET            0.030695
INST_TYPE_1       0.014885
INST_TYPE_2       0.012420
HIGH_DEG          0.010169
INST_TYPE_3       0.003288
dtype: float64


In [56]:
# #### Re-training model by using significant features
# X = dataset[['ST_FI_H2','HL_ED_P_HS','PER_PT','INST_EXP_PS','AVG_FAC_SAL_PM','TS','ST_FI_M1','TS_MAR','TS_WOM','ST_FI_H1','HL_ED_P_MS']]

# feature_scaler = StandardScaler()
# X_scaled = feature_scaler.fit_transform(X)

# rfr = RandomForestRegressor(criterion='mse', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [900,950,1000,1050,1100]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# print("Cross-validation results:\n", pd.DataFrame.from_dict(gd_sr.cv_results_))

# best_parameters = gd_sr.best_params_
# print("Optimal parameters:\n", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("Best mean cross-validated score:\n", best_result)

Cross-validation results:
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0      41.937993      0.168599         1.003917        0.053506   
1      43.833799      0.221262         1.050106        0.077963   
2      47.450242      0.223039         1.207211        0.098758   

  param_n_estimators                  params  split0_test_score  \
0               1000  {'n_estimators': 1000}           0.611165   
1               1050  {'n_estimators': 1050}           0.611206   
2               1100  {'n_estimators': 1100}           0.611075   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.652725           0.704582           0.743014           0.683642   
1           0.652665           0.704478           0.742809           0.683701   
2           0.652842           0.704404           0.742798           0.683690   

   mean_test_score  std_test_score  rank_test_score  
0         0.679025        0.044868                1  
1 

In [60]:
sgdr = SGDRegressor(random_state = 1)
grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[5000,6000,8000,9000,10000,20000]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print(best_parameters)

best_result = gd_sr.best_score_ 
print(best_result)

{'eta0': 0.01, 'max_iter': 5000}
0.47859535445861867


In [59]:
sgdr = SGDRegressor(eta0=.001, max_iter=5000, penalty=None, random_state=1)
sgdr.fit(X_scaled,Y)
print('Intercept', sgdr.intercept_)
print(pd.DataFrame(zip(X.columns, sgdr.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Intercept [0.50952636]
          Features  Coefficients
0         ST_FI_H2      0.055472
3      INST_EXP_PS      0.034785
7           TS_MAR      0.025774
10      HL_ED_P_MS      0.017926
5               TS      0.014213
8           TS_WOM      0.006084
4   AVG_FAC_SAL_PM      0.002824
9         ST_FI_H1     -0.003782
6         ST_FI_M1     -0.016501
1       HL_ED_P_HS     -0.021176
2           PER_PT     -0.024676
